In [1]:
import json
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureSet
from arcgis.geometry import Geometry
from arcgis.geometry import SpatialReference
import tempfile
import os
import math 

In [2]:
gis = GIS()
print(f"Connected to: {gis}")

Connected to: GIS @ https://www.arcgis.com version:2025.2


In [3]:
def load_geojson_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

lands_data = load_geojson_from_file('land.geojson')
boundary_data = load_geojson_from_file('boundary.geojson')

In [4]:
map = gis.map('Bangkok') 
map.zoom = 15
map.center = [100.5535, 13.7585]
map

Map(center=[100.5535, 13.7585], extent={'xmin': 11158024.913073143, 'ymin': 1515306.9157138057, 'xmax': 112176…

In [5]:
def parse_land_features(lands_data):
    land_features = []
    features = lands_data['features']
    for feature in features:  
        geometry = Geometry(feature['geometry'])
        attributes = feature['properties']
        
        arcgis_feature = {
            'geometry': geometry,
            'attributes': attributes
        }
        land_features.append(arcgis_feature) 
        
    spatial_ref = SpatialReference(wkid=24047)
    land_feature_set = FeatureSet(land_features, spatial_reference=spatial_ref)  # Corrected list name
    return land_features, land_feature_set


land_features, land_feature_set = parse_land_features(lands_data)
map.content.add(land_feature_set)



In [6]:
def parse_boundary_feature(boundary_data):
    boundary_feature = []
    for feature in boundary_data['features']:
        geometry = Geometry(feature['geometry'])
        attributes = feature['properties']
        
        arcgis_feature = {
            'geometry': geometry,
            'attributes': attributes
        }
        boundary_feature.append(arcgis_feature)
    spatial_ref = SpatialReference(wkid=24047)
    boundary_feature_set = FeatureSet(boundary_feature, spatial_reference=spatial_ref)
        
    return boundary_feature, boundary_feature_set

boundary_feature, boundary_feature_set = parse_boundary_feature(boundary_data)
# map.content.add(boundary_feature_set)  
    

In [7]:
def min_area_parcel(lands):
    min_area = lands[0]['geometry'].area
    for land in lands: 
        geometry = land['geometry']
        area = geometry.area
        if area < min_area:
            min_area = area 
    return min_area 

min_area = min_area_parcel(land_features)
min_area

19431.005550002472

In [8]:
import math 
from arcgis.geometry import Point, Polyline, Polygon

def generate_grid(boundary, lands):
    min_area = min_area_parcel(lands)
    cell_size = math.sqrt(min_area) 
    borders = [] 
    boundary_extent = boundary[0]['geometry'].extent
    min_x, min_y, max_x, max_y = boundary_extent[0], boundary_extent[1], boundary_extent[2], boundary_extent[3]
    cols = int(math.ceil((max_x - min_x) / cell_size))
    rows = int(math.ceil((max_y - min_y) / cell_size))

    # Horizontal segments (between cells vertically)
    for row in range(rows + 1):
        y = min_y + row * cell_size
        for col in range(cols):
            x1 = min_x + col * cell_size
            x2 = min_x + (col + 1) * cell_size
            
            border_line = Polyline({
                'paths': [[[x1, y], [x2, y]]],
                'spatialReference': {'wkid': 24047}
            })
            borders.append(border_line)
    
    # Vertical segments (between cells horizontally)  
    for col in range(cols + 1):
        x = min_x + col * cell_size
        for row in range(rows):
            y1 = min_y + row * cell_size
            y2 = min_y + (row + 1) * cell_size
            
            border_line = Polyline({
                'paths': [[[x, y1], [x, y2]]],
                'spatialReference': {'wkid': 24047}
            })
            borders.append(border_line)
    
    return borders 

grid_borders = generate_grid(boundary_feature, land_features)

In [9]:
from arcgis.geometry import buffer, LengthUnits, Geometry
from arcgis.geometry import functions as geom_functions
from arcgis.geometry import SpatialReference
from arcgis.features import FeatureSet

def calculate_road_impact(road, lands):
    connected_lands = [] 
    total_price = 0
    for land in lands:
        intersect_geom = geom_functions.intersect(24047, [land['geometry']], road)[0]
        land_id = land['attributes']['id']
        price_per_sqm = land['attributes']['price_per_sqm']
        area = intersect_geom.area
        if area > 0:
            total_price += area * price_per_sqm
            connected_lands.append(land_id)
    return connected_lands, total_price 

def generate_road_candidate(borders, lands):
    road_features = [] 
    for i, border in enumerate(borders):
        buffer_result = buffer(geometries=[border], distances=[12], in_sr={"wkid": 24047}, unit=LengthUnits.METER)
        road = Geometry(buffer_result[0])

        connected_lands, road_price = calculate_road_impact(road, lands)
        print(f"id: {i} road price: {road_price}, connected_lands: {connected_lands}")

        arcgis_feature = {
            'geometry': road,
            'attributes': {
                'id': i,
                'road_price': road_price,
                'connected_lands': connected_lands,
                'efficiency': float('inf')
            }
        }
        if len(connected_lands) > 0:
            road_features.append(arcgis_feature) 

    return road_features 


In [10]:
road_features = generate_road_candidate(grid_borders, land_features)

id: 0 road price: 0, connected_lands: []
id: 1 road price: 0, connected_lands: []
id: 2 road price: 0, connected_lands: []
id: 3 road price: 0, connected_lands: []
id: 4 road price: 0, connected_lands: []
id: 5 road price: 0, connected_lands: []
id: 6 road price: 0, connected_lands: []
id: 7 road price: 0, connected_lands: []
id: 8 road price: 0, connected_lands: []
id: 9 road price: 87856173.65113384, connected_lands: [1]
id: 10 road price: 213501412.30232453, connected_lands: [1]
id: 11 road price: 223312456.0454561, connected_lands: [1, 2]
id: 12 road price: 227734839.78903696, connected_lands: [2]
id: 13 road price: 199841393.8419662, connected_lands: [2, 3]
id: 14 road price: 113867419.89457308, connected_lands: [3]
id: 15 road price: 185832906.18386796, connected_lands: [3, 4]
id: 16 road price: 190082108.2364117, connected_lands: [4]
id: 17 road price: 6339811.209999487, connected_lands: [4]
id: 18 road price: 23068759.32700491, connected_lands: [1, 5]
id: 19 road price: 1887608

KeyboardInterrupt: 

In [ ]:
def find_connected_roads_simple(roads, boundary, min_area):
    # Calculate grid dimensions
    cell_size = math.sqrt(min_area)
    boundary_extent = boundary[0]['geometry'].extent
    min_x, min_y, max_x, max_y = boundary_extent[0], boundary_extent[1], boundary_extent[2], boundary_extent[3]
    n_cols = int(math.ceil((max_x - min_x) / cell_size))
    n_rows = int(math.ceil((max_y - min_y) / cell_size))
    
    horizontal_count = (n_rows + 1) * n_cols
    
    for i, road in enumerate(roads):
        connected_roads = []
        
        if i < horizontal_count:
            # Horizontal border - connects to 4 vertical borders (if they exist)
            row = i // n_cols
            col = i % n_cols
            
            # Potential connections
            potential_connections = []
            if row > 0:
                potential_connections.extend([
                    horizontal_count + (row - 1) * (n_cols + 1) + col,      # left-above
                    horizontal_count + (row - 1) * (n_cols + 1) + col + 1   # right-above
                ])
            if row < n_rows:
                potential_connections.extend([
                    horizontal_count + row * (n_cols + 1) + col,            # left-below
                    horizontal_count + row * (n_cols + 1) + col + 1         # right-below
                ])
            
            # Filter valid connections
            connected_roads = [c for c in potential_connections if 0 <= c < len(roads)]
            
        else:
            # Vertical border - connects to 4 horizontal borders (if they exist)
            vertical_index = i - horizontal_count
            row = vertical_index // (n_cols + 1)
            col = vertical_index % (n_cols + 1)
            
            # Potential connections
            potential_connections = []
            if col > 0:
                potential_connections.extend([
                    row * n_cols + (col - 1),        # top-left
                    (row + 1) * n_cols + (col - 1)   # bottom-left
                ])
            if col < n_cols:
                potential_connections.extend([
                    row * n_cols + col,              # top-right
                    (row + 1) * n_cols + col         # bottom-right
                ])
            
            # Filter valid connections
            connected_roads = [c for c in potential_connections if 0 <= c < len(roads)]
        
        road['attributes']['connected_roads'] = connected_roads
        print(f"road {i}: connected_roads: {connected_roads}")
    
    return roads

In [ ]:
update_road_features = find_connected_roads(road_features, road_features)  # Pass the same list as possible_roads
spatial_ref = SpatialReference(wkid=24047)
road_feature_set = FeatureSet(update_road_features, spatial_reference=spatial_ref)

In [ ]:
def calculate_road_efficiency(road_features, connected_land_ids):
    for road in road_network:
        road_connected_lands = road['attributes']['connected_lands']
        unconnected_land_count = 0
        for land_id in road_connected_lands:
            if land_id not in connected_land_ids:
                unconnected_land_count += 1
                
        if unconnected_land_count > 0:
            road['attributes']['efficiency'] = road['attributes']['road_price'] / unconnected_land_count
        else:
            road['attributes']['efficiency'] = float('inf')

connected_land_ids = [] 
selected_roads = [] 

while len(connected_land_ids) < 20:
    # Calculate the efficiency of each road
    calculate_road_efficiency(road_features, connected_land_ids)
    
    # Filter out roads that can't connect any new land
    candidate_roads = [road for road in road_features if road['attributes']['efficiency'] != float('inf')]
    
    if not candidate_roads:
        print("No more roads can connect new lands!")
        break
    
    # Sort roads by efficiency (lowest first)
    candidate_roads.sort(key=lambda x: x['attributes']['efficiency'])
    
    # Select the road with the highest efficiency
    selected_road = candidate_roads[0]
    selected_roads.append(selected_road)
    
    # Add the lands connected by the selected road to the connected land list
    for land_id in selected_road['attributes']['connected_lands']:
        if land_id not in connected_land_ids:
            connected_land_ids.append(land_id)
    
    # Remove the selected road from the list of roads
    road_features.remove(selected_road)


In [ ]:
update_road_features = valid_roads 
spatial_ref = SpatialReference(wkid=24047)
update_road_feature_set = FeatureSet(update_road_features, spatial_reference=spatial_ref)


In [ ]:
map.content.add(update_road_feature_set)

In [ ]:
road_features

In [ ]:
map